In [16]:
'''import necessary libraries'''
import pandas as pd
import pypyodbc
import sys
import os
import sqlalchemy
import numpy as np

'''Input folder from where the file will be picked up'''
input_folder_path='D:\\Pen drive new\\IISC proficiency\\Basics of Data Analytics\\Data Input Module\\Input\\'


'''Function#1: to read csv files from the Input folder and create corrosponding tables in the database if the file satisfies data type checks'''
def create_tables_from_files():
    input_file_list=os.listdir(input_folder_path)
    for f in input_file_list:
        file_name=f[:len(f)-4]
        exist_flag=chk_table_name_exist(table_name=file_name)
        if(exist_flag==1):
            print('file',f,'not loaded to database as table with the same filename exists. Please supply a different file name or delete the existing table.')
            continue
        else:
            fq_file_name=input_folder_path+f
            df=pd.read_csv(fq_file_name)
            '''check the datatypes of the file columns, if any non-int or non-float type are present display error'''
            type_flag=chk_dtype(df)
            if(type_flag==1):
                print('file',f,'not loaded as it contains non-int or non-float datatypes')
                continue
            load_df_to_table(df,file_name)
            create_outlier_column(file_name)
            
            
'''Function#2: to check if a table with the same name as the file exists'''
def chk_table_name_exist(table_name):
    '''Open connection to SQL database'''
    connection = pypyodbc.connect('Driver={SQL Server};''Server=DESKTOP-FPH6QIE\SQLEXPRESS;''Database=DataMart;''Trusted_Connection=Yes')
    cursor=connection.cursor() 
    SQLCommand = 'SELECT COUNT(1) as table_exist FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME=?'
    values=[table_name]
    cursor.execute(SQLCommand,values)
    results=cursor.fetchone()
    table_exist=int(str(results).replace(',','').replace('(','').replace(')',''))
    '''close connection to SQL database'''
    connection.close()
    if(table_exist>0):
        return 1
    else:
        return 0
    
    
'''Function#3: to check if any of the file columns are not integer or float'''
def chk_dtype(df):
    i=0
    for col in df.columns:
        t=str(df[col].dtype)
        if((t=='float64') or (t=='int64')):
            i+=0
        else:
            i+=1
    if(i>0):
        return 1
    else:
        return 0
    
'''Function#4: to insert dataframe rows to sql table'''
def load_df_to_table(df,table_name):
    engine = sqlalchemy.create_engine('mssql://DESKTOP-FPH6QIE\SQLEXPRESS/DataMart?driver=SQL+Server+Native+Client+11.0')
    df.to_sql(table_name,engine,schema='dbo',if_exists='fail',index=False)
    
    
'''function#5: to view the list of tables and columns in the database'''
def display_tables_and_columns():
    '''Open connection to SQL database'''
    connection = pypyodbc.connect('Driver={SQL Server};''Server=DESKTOP-FPH6QIE\SQLEXPRESS;''Database=DataMart;''Trusted_Connection=Yes')
    cursor=connection.cursor() 
    SQLCommand="SELECT table_name, column_names= STUFF((SELECT ', ' + column_name FROM INFORMATION_SCHEMA.COLUMNS b WHERE b.table_name = a.table_name FOR XML PATH('')), 1, 2, '') FROM INFORMATION_SCHEMA.COLUMNS a GROUP BY table_name"
    cursor.execute(SQLCommand)
    results=cursor.fetchall()
    print(results)
    '''Close SQL connection'''
    connection.close()
    
    
'''Function#6: to create the  additional "outlier" column to identify and mark outliers among the data'''
def create_outlier_column(table_name):
    '''Open connection to SQL database'''
    connection = pypyodbc.connect('Driver={SQL Server};''Server=DESKTOP-FPH6QIE\SQLEXPRESS;''Database=DataMart;''Trusted_Connection=Yes')
    cursor=connection.cursor()
    SQLCommand='alter table '+table_name+' add [outlier] float not null default 0'
    cursor.execute(SQLCommand)
    '''Commit and close SQL connection'''
    connection.commit()
    connection.close()
    
    
'''Function#7: to retrieve data from a table into an array'''
def get_data(table_name,outlier):
    engine = sqlalchemy.create_engine('mssql://DESKTOP-FPH6QIE\SQLEXPRESS/DataMart?driver=SQL+Server+Native+Client+11.0')
    col_str=get_column_of_table(table_name)
    if(outlier==0):
        SQLCommand="select "+col_str+' from '+table_name+' where outlier='+str(outlier)
    else:
        SQLCommand="select "+col_str+' from '+table_name
    op=pd.read_sql(SQLCommand,engine,index_col=None)
    array=op.values
    return array

'''Function#8: to obtain as a comma seperated list, the columns of a table'''
def get_column_of_table(table_name):
    connection = pypyodbc.connect('Driver={SQL Server};''Server=DESKTOP-FPH6QIE\SQLEXPRESS;''Database=DataMart;''Trusted_Connection=Yes')
    cursor=connection.cursor() 
    SQLCommand="SELECT STUFF((SELECT ',' + '['+column_name+']' FROM INFORMATION_SCHEMA.COLUMNS where table_name='"+table_name+"' and COLUMN_NAME <>'outlier' FOR XML PATH('')), 1, 1, '') As [output]"
    cursor.execute(SQLCommand)
    results=cursor.fetchone()
    col_str=list(results)[0]
    return col_str
    connection.close()
    
    
'''Function#9: to update records as outliers'''
def update_outlier(outlier_array,table_name):
    drop_table('outlier_table')
    '''Open connection to SQL database'''
    connection = pypyodbc.connect('Driver={SQL Server};''Server=DESKTOP-FPH6QIE\SQLEXPRESS;''Database=DataMart;''Trusted_Connection=Yes')
    cursor=connection.cursor() 
    col_list_df=get_column_of_table('Iris_dataset').replace('[','').replace(']','').split(',')
    col_list=get_column_of_table('Iris_dataset').split(',')
    a_col_list=['a.'+x for x in col_list]
    b_col_list=['b.'+x for x in col_list]
    join_col_list=[x+'='+y for x,y in zip(a_col_list,b_col_list)]
    join_col_string = ",".join(join_col_list )
    join_col_string=join_col_string.replace(',',' and ')
    outlier_df=pd.DataFrame(outlier_array,columns=col_list_df)
    outlier_df.head()
    load_df_to_table(outlier_df,'outlier_table')
    SQLCommand="update a set a.outlier=1 from "+table_name+" a join outlier_table b on "+join_col_string
    cursor.execute(SQLCommand)
    '''commit and close connection'''
    connection.commit()
    connection.close()
    
    
'''Function#10 that permanantly drops a table from the database if it exists'''
def drop_table(table_name):
    '''Open connection to SQL database'''
    connection = pypyodbc.connect('Driver={SQL Server};''Server=DESKTOP-FPH6QIE\SQLEXPRESS;''Database=DataMart;''Trusted_Connection=Yes')
    cursor=connection.cursor() 
    SQLCommand="IF OBJECT_ID('dbo."+table_name+"', 'U') IS NOT NULL DROP TABLE dbo."+table_name
    cursor.execute(SQLCommand)
    '''commit and close connection'''
    connection.commit()
    connection.close()
